# Module 3: Camera and Computer Vision Basics

Now that we have basic motion and odometery implemented, we will look at the camera on the <span style="color:#154734">Jetbot</span> and how images are handled within Python.

This Module should follow Module 2: Path Following

As always, we need to first initialize some of the packages and objects for our <span style="color:#154734">Jetbot</span>.

In [ ]:
# sudo pip install --upgrade pip
# sudo pip install opencv-python

In [ ]:
from jetbot import Robot, Camera, bgr8_to_jpeg
import cv2
import numpy as np
import ipywidgets as widgets
from IPython.display import display, Image, clear_output
import time

robot = Robot()

There are now a few new items we are importing into our code now. In no particular order, they are:
- from Jetbot import Camera, bgr8_to_jpeg
- import cv2
- import numpy as np
- from IPython.display import display, Image, clear_output
- import time

Each item is important to making our camera function and display, while also giving us the ability to modify our image to our liking. Their functions and usage will be described following each of the code blocks below.

### Taking Pictures

In [ ]:
# Initialize our camera
camera = Camera.instance()

# Save image data (modified to be numpy array)
image = np.array(camera.value)

jpeg_image = bgr8_to_jpeg(image)

# Display image within Jupyter Notebook
display(Image(data=jpeg_image))

This is our basic method of taking an image and displaying it within Jupyter Notebook.

First, we need to initialize our camera, as shown in the first line. When this is done, the image that the camera takes is stored in *camera.value*.

Knowing where the image is stored, we can assign that to a variable as shown in the second line. Additionally, this value is stored as a bgr8 file (standing for blue, green, red, 8 bit), and the np.array function basically simplifies the way it is stored (in an easier to understand format).

However, to display the image, we need to store it as a jpeg image. Luckily, jetbot has a function for this labeled as bgr8_to_jpeg. By calling this function with the image as an argument, we now have an image stored as a JPEG.

Lastly, we need to display the image in Jupyter Notebook. One of the ways we display any outputs is using the display() command. But simply passing jpeg_image to display will print the data as it is stored (with numbers and characters), and not as a picture. Therefore, we need the Image() command, and pass the jpeg_image to the argument data within.

This will only output one picture, since the block is only being run once. If we want to continuously output a live feed from the camera, we can implement these features in a loop as shown below.

<div class="alert alert-block alert-info">
Opportunity for Activity
</div>

In [ ]:
while True:
    clear_output(wait = True)
    image = np.array(camera.value)
    jpeg_image = bgr8_to_jpeg(image)
    display(Image(data = jpeg_image))
    time.sleep(0.5)

> Note that the time.sleep(0.5) is used to set the frame rate, where a new image is displayed every 0.5 seconds. To make it faster, we can decrease the time within, effectively increasing the frequency at which the image updates.

In addition, we now need to include clear_output(wait = True). This command will replace the image below the block, so that it won't constantly display new pictures (You can see the effects of what happens if you remove that command).

### Editing/Modifying Images

Now for one of the most important part of the lessons for this module, is how to operate computer vision techniques on the images. This will be primarily using opencv, otherwise known as cv2. The code below shows some options for what can be accomplished

<div class="alert alert-block alert-danger">
You will likely need to interrupt your kernel before you can continue with the square block above (otherwise it will be stuck in the loop).
</div>

In [ ]:
def process_and_display_image():
    image = np.array(camera.value)
    gray_image = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    display(Image(data=bgr8_to_jpeg(gray_image)))

Similar to before, we must save our image from the camera as a numpy array. But now, we will perform operations on the image before we display them.

One such operation is changing the image to grayscale. As seen above, the code takes the image, and modifies it from RGB to Gray. Afterwards, we can display the image (after transforming it to a jpeg) and you will see the image from your camera in gray.

Similar to before, we can create a live feed using a loop again.

<div class="alert alert-block alert-info">
Opportunity for Activity
</div>

In [ ]:
while True:
    clear_output(wait=True)
    process_and_display_image()
    time.sleep(0.01)

Lastly, this can also be accomplished via threading, which allows it to run simultaneousy with other code. But to do it this way, it needs to be in a jupyter notebook widget.

In [ ]:
import threading
image = widgets.Image(format='jpeg', width=300, height=300)
display(image)

def update_image():
    while True:
        image.value = bgr8_to_jpeg(camera.value)
        time.sleep(0.01)

threading.Thread(target=update_image, daemon=True).start()

<div class="alert alert-block alert-info">
Will probably need to go more in depth on computer vision and other functions that are possible, but don't really want to do it right now. But likely could look at masking/color detection and edge detection in preparation for next module.
</div>

### Camera Calibration

If you'll notice on the images being displayed, there is quite a bit of distortion in the image. It becomes especially clear if you hold something with a straight edge in front of it, like a piece of paper, the camera completely bends and curves the edge.

In [ ]:

# termination criteria
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)

# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((4*7,3), np.float32)
objp[:,:2] = np.mgrid[0:7,0:4].T.reshape(-1,2)
objp *= 0.035

# Arrays to store object points and image points from all the images.
objpoints = [] # 3d point in real world space
imgpoints = [] # 2d points in image plane.

count = 0

while count < 10:
    clear_output(wait=True)

    img = camera.value
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Find the chess board corners
    ret, corners = cv2.findChessboardCorners(gray, (7,4), None)

    # If found, add object points, image points (after refining them)
    if ret == True:
        objpoints.append(objp)

        corners2 = cv2.cornerSubPix(gray,corners, (11,11), (-1,-1), criteria)
        imgpoints.append(corners2)

        # Draw and display the corners
        cv2.drawChessboardCorners(img, (7,4), corners2, ret)
    display(Image(data=bgr8_to_jpeg(img)))
    time.sleep(0.01)




In [ ]:
import ipywidgets as widgets
from IPython.display import display
import threading
import time
import os

# 1. Checkerboard Parameters (inner corners, square size in meters)
CHECKERBOARD = (7, 5)  # 7 columns, 5 rows of inner corners
SQUARE_SIZE = 0.024  # in meters (e.g., 24mm squares)

# termination criteria
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)

# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((CHECKERBOARD[0]*CHECKERBOARD[1],3), np.float32)
objp[:,:2] = np.mgrid[0:CHECKERBOARD[0],0:CHECKERBOARD[1]].T.reshape(-1,2)
objp *= SQUARE_SIZE

# Arrays to store object points and image points from all the images.
objpoints = [] # 3d point in real world space
imgpoints = [] # 2d points in image plane.

# 1. Setup Widgets
image = widgets.Image(format='jpeg', width=480, height=360)
save_button = widgets.Button(description='📸 Save Image')
saved_count = 0
status_label = widgets.Label(value='Ready')
display(widgets.VBox([image, save_button, status_label]))



# 3. Image Update Function (Threaded)
def update_image_loop():
    while True:
        frame = camera.value
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        ret, corners = cv2.findChessboardCorners(gray, CHECKERBOARD, None)
        # If found, add object points, image points (after refining them)
        if ret == True:
            # objpoints.append(objp)

            corners2 = cv2.cornerSubPix(gray,corners, (11,11), (-1,-1), criteria)
            # imgpoints.append(corners2)

            # Draw and display the corners
            cv2.drawChessboardCorners(frame, CHECKERBOARD, corners2, ret)
            image.value = bgr8_to_jpeg(frame)
        time.sleep(0.01)  # control the update rate

# 3. Button Callback to Save Image
SAVE_DIR = "calibration_images"
os.makedirs(SAVE_DIR, exist_ok=True)

def save_image_callback(b):
    global saved_count
    if image.value:
        filename = f"image_{saved_count}.jpg"
        filepath = os.path.join(SAVE_DIR, filename)
        with open(filepath, 'wb') as f:
            f.write(image.value)
        status_label.value = f"✅ Saved {filename}"
        saved_count += 1
    else:
        status_label.value = "⚠️ No image available."

# 4. Connect Callback
save_button.on_click(save_image_callback)

# 5. Start Image Update Thread (non-blocking)
threading.Thread(target=update_image_loop, daemon=True).start()


In [ ]:
# 📦 Imports
import os
import io
import numpy as np
import cv2
from datetime import datetime
from PIL import Image
import ipywidgets as widgets
from IPython.display import display

# 📁 Directory to store calibration images
SAVE_DIR = "calib_images"
os.makedirs(SAVE_DIR, exist_ok=True)

# 📐 Checkerboard settings (inner corners, square size in meters)
CHECKERBOARD = (7, 5)  # 7 columns, 4 rows of inner corners
SQUARE_SIZE = 0.024  # in meters (e.g., 24mm squares)

# 🎞️ Live image widget (assumes this is being updated elsewhere)
image = widgets.Image(format='jpeg', width=480, height=360)

# 📸 Save Button
save_button = widgets.Button(description="📸 Save Image")
save_counter = widgets.Label(value="Saved: 0")
saved_count = 0

# 💾 Save function
def on_save_clicked(b):
    global saved_count
    if image.value:
        filename = f"calib_{saved_count}.jpg"
        filepath = os.path.join(SAVE_DIR, filename)
        with open(filepath, 'wb') as f:
            f.write(image.value)
        saved_count += 1
        save_counter.value = f"Saved: {saved_count}"
        print(f"✅ Saved {filepath}")

save_button.on_click(on_save_clicked)

# 📺 Display the UI
display(widgets.VBox([image, widgets.HBox([save_button, save_counter])]))
